In [8]:
#data source
#https://search.earthdata.nasa.gov/search/granules?p=C1940475563-POCLOUD&pg[0][v]=f&pg[0][gsk]=start_date&g=G1996941394-POCLOUD&tl=1666010677.42!3!!&lat=78.609375&long=0.140625&zoom=1


https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MODIS_T-JPL-L2P-v2019.0/20000224003506-JPL-L2P_GHRSST-SSTskin-MODIS_T-D-v02.0-fv01.0.nc
#https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MODIS_T-JPL-L2P-v2019.0/20000224000006-JPL-L2P_GHRSST-SSTskin-MODIS_T-N-v02.0-fv01.0.nc


#https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html


SyntaxError: invalid syntax (<ipython-input-8-06a2b411352e>, line 5)

In [9]:

from ipynb.fs.full.utils import *
from glob import glob
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re

import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt



In [7]:

fn = '/home/fun/sst/sst.day.mean.2020.v2.nc'
ds = nc.Dataset(fn)


print(ds)
#print(ds.__dict__)
# for dim in ds.dimensions.values():
#     print(dim)
    
# for var in ds.variables.values():
#     print(var)
    
    
#print(ds['sst'])



sst.shape


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.5
    title: NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surface Temperature (OISST) Analysis, Version 2.1
    institution: NOAA/National Centers for Environmental Information
    source: NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/
    References: https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html
    dataset_title: NOAA Daily Optimum Interpolation Sea Surface Temperature
    version: Version 2.1
    comment: Reynolds, et al.(2007) Daily High-Resolution-Blended Analyses for Sea Surface Temperature (available at https://doi.org/10.1175/2007JCLI1824.1). Banzon, et al.(2016) A long-term record of blended satellite and in situ sea-surface temperature for climate monitoring, modeling and environmental studies (available at https://doi.org/10.5194/essd-8-165-2016). Huang et al. (2020) Impr

NameError: name 'sst' is not defined

In [8]:
plt.imshow(sst[250])
plt.gca().invert_yaxis()

NameError: name 'sst' is not defined

In [10]:
def date_to_nth_day(date, fmt='%Y%m%d'):
    date = datetime.datetime.strptime(date, fmt)
    new_year_day = datetime.datetime(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1


def getTgtArea(rgb, lon, lat, radias):
    '''
    lat (-90, 90)
    lon (0, E(0, 180), W(0, 180))
    '''
    n = 720
    m = 1440
    
    interval = 180.0 / n
    
    lat_index = int((90 + lat) / interval)

    long_index = int((lon + 360) / interval) # only works for West
    
    n_raidus = int(radias / interval)
    
    
    return rgb[lat_index - n_raidus:lat_index + n_raidus, long_index - n_raidus:long_index + n_raidus]
def generate_noaa_filename(datestr):
    '''
    compose url using date  'YYYYMMDD'
    '''
    fname = '/home/fun/sst/sst.day.mean.' + datestr[0:4] + '.v2.nc'
    return fname
    #datestr[4:6] + '/' + datestr[6:8]
    
def post_processing(folder, sst, clip):
    idx = date_to_nth_day(str(clip['date']))  
    area = getTgtArea(sst[idx], -1 * float(clip['lg'][:-1]), float(clip['lt'][:-1]), 12.5)
    print(type(area))
    
    final = np.array([area])
    pathname = folder + '/noaa_sst_' + str(clip['date']) + '.npy'
    np.save(pathname, final)
    
#   call it noaa_sst_
    
   
def download_noaa(folder):
    json_profile = folder + '/profile.json'
    json_file = open(json_profile)
    d = json.load(json_file)
        
    for it in d['path']:
        print(it)
        fname = generate_noaa_filename(str(it['date']))
        
        ds = nc.Dataset(fname)
        sst = ds['sst']
        post_processing(folder, sst, it)
        


In [11]:

hurricanes_folders = glob("/home/fun/data/AL*20*", recursive = True)


for it in hurricanes_folders:
    sst = download_noaa(it)


{'date': 20040803, 'time': '1200', 'category': 'TD', 'lt': '12.9N', 'lg': '53.6W', 'maxwind': '25', 'maxpres': '1010', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040803, 'time': '1800', 'category': 'TD', 'lt': '13.2N', 'lg': '55.4W', 'maxwind': '25', 'maxpres': '1010', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040804, 'time': '0000', 'category': 'TD', 'lt': '13.5N', 'lg': '57.4W', 'maxwind': '30', 'maxpres': '1010', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '6

{'date': 20040809, 'time': '1200', 'category': 'TS', 'lt': '22.5N', 'lg': '87.6W', 'maxwind': '35', 'maxpres': '1008', '34ktne': '25', '34ktse': '25', '34ktsw': '25', '34ktnw': '25', '50ktne': '25', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040809, 'time': '1800', 'category': 'TS', 'lt': '22.9N', 'lg': '88.3W', 'maxwind': '40', 'maxpres': '1007', '34ktne': '25', '34ktse': '25', '34ktsw': '25', '34ktnw': '25', '50ktne': '25', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040810, 'time': '0000', 'category': 'TS', 'lt': '23.1N', 'lg': '89.0W', 'maxwind': '45', 'maxpres': '1006', '34ktne': '25', '34ktse': '25', '34ktsw': '25', '34ktnw': '25', '50ktne': '25', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '6

<class 'numpy.ma.core.MaskedArray'>
{'date': 20040815, 'time': '0600', 'category': 'HU', 'lt': '14.1N', 'lg': '30.8W', 'maxwind': '75', 'maxpres': '981', '34ktne': '120', '34ktse': '75', '34ktsw': '75', '34ktnw': '75', '50ktne': '100', '50ktse': '50', '50ktsw': '40', '50ktnw': '40', '64ktne': '50', '64ktse': '25', '64ktsw': '20', '64ktnw': '20', 'maxwindradius': '25'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040815, 'time': '1200', 'category': 'HU', 'lt': '14.7N', 'lg': '32.1W', 'maxwind': '80', 'maxpres': '978', '34ktne': '120', '34ktse': '75', '34ktsw': '75', '34ktnw': '75', '50ktne': '120', '50ktse': '75', '50ktsw': '40', '50ktnw': '40', '64ktne': '75', '64ktse': '25', '64ktsw': '20', '64ktnw': '20', 'maxwindradius': '25'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040815, 'time': '1800', 'category': 'HU', 'lt': '15.2N', 'lg': '33.5W', 'maxwind': '85', 'maxpres': '975', '34ktne': '120', '34ktse': '75', '34ktsw': '75', '34ktnw': '75', '50ktne': '120', '50ktse': '60', '50kts

<class 'numpy.ma.core.MaskedArray'>
{'date': 20040821, 'time': '1200', 'category': 'TD', 'lt': '30.7N', 'lg': '38.9W', 'maxwind': '25', 'maxpres': '1014', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040821, 'time': '1800', 'category': 'LO', 'lt': '30.5N', 'lg': '39.2W', 'maxwind': '25', 'maxpres': '1014', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040822, 'time': '0000', 'category': 'LO', 'lt': '30.3N', 'lg': '39.8W', 'maxwind': '25', 'maxpres': '1015', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '

<class 'numpy.ma.core.MaskedArray'>
{'date': 20040827, 'time': '1800', 'category': 'TD', 'lt': '31.6N', 'lg': '78.0W', 'maxwind': '25', 'maxpres': '1014', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040828, 'time': '0000', 'category': 'TD', 'lt': '31.4N', 'lg': '78.0W', 'maxwind': '30', 'maxpres': '1011', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20040828, 'time': '0600', 'category': 'TS', 'lt': '31.3N', 'lg': '78.2W', 'maxwind': '35', 'maxpres': '1005', '34ktne': '0', '34ktse': '50', '34ktsw': '50', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne':

...

{'date': 20051119, 'time': '0000', 'category': 'TS', 'lt': '16.0N', 'lg': '85.6W', 'maxwind': '40', 'maxpres': '1005', '34ktne': '90', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '90', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051119, 'time': '0600', 'category': 'TS', 'lt': '16.1N', 'lg': '85.6W', 'maxwind': '40', 'maxpres': '1004', '34ktne': '90', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '90', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051119, 'time': '1200', 'category': 'TS', 'lt': '16.1N', 'lg': '85.4W', 'maxwind': '45', 'maxpres': '1002', '34ktne': '90', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '90', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '

<class 'numpy.ma.core.MaskedArray'>
{'date': 20051123, 'time': '0600', 'category': 'SS', 'lt': '28.8N', 'lg': '41.3W', 'maxwind': '45', 'maxpres': '983', '34ktne': '250', '34ktse': '200', '34ktsw': '200', '34ktnw': '225', '50ktne': '275', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051123, 'time': '1200', 'category': 'TS', 'lt': '27.4N', 'lg': '41.2W', 'maxwind': '50', 'maxpres': '983', '34ktne': '250', '34ktse': '200', '34ktsw': '200', '34ktnw': '250', '50ktne': '275', '50ktse': '45', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051123, 'time': '1800', 'category': 'TS', 'lt': '26.4N', 'lg': '40.8W', 'maxwind': '50', 'maxpres': '983', '34ktne': '200', '34ktse': '175', '34ktsw': '175', '34ktnw': '225', '50ktne': '250', '50ktse': '50', '50ktsw': '3

<class 'numpy.ma.core.MaskedArray'>
{'date': 20051129, 'time': '1200', 'category': 'EX', 'lt': '32.6N', 'lg': '6.6W', 'maxwind': '35', 'maxpres': '1000', '34ktne': '0', '34ktse': '150', '34ktsw': '150', '34ktnw': '150', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051129, 'time': '1800', 'category': 'EX', 'lt': '35.3N', 'lg': '1.0W', 'maxwind': '30', 'maxpres': '1003', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051129, 'time': '0600', 'category': 'TS', 'lt': '31.5N', 'lg': '49.2W', 'maxwind': '45', 'maxpres': '993', '34ktne': '45', '34ktse': '0', '34ktsw': '0', '34ktnw': '90', '50ktne': '120', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64k

<class 'numpy.ma.core.MaskedArray'>
{'date': 20051205, 'time': '0600', 'category': 'HU', 'lt': '34.0N', 'lg': '36.7W', 'maxwind': '75', 'maxpres': '981', '34ktne': '75', '34ktse': '75', '34ktsw': '75', '34ktnw': '100', '50ktne': '75', '50ktse': '30', '50ktsw': '30', '50ktnw': '50', '64ktne': '30', '64ktse': '0', '64ktsw': '20', '64ktnw': '20', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051205, 'time': '1200', 'category': 'HU', 'lt': '33.8N', 'lg': '35.5W', 'maxwind': '75', 'maxpres': '981', '34ktne': '75', '34ktse': '75', '34ktsw': '75', '34ktnw': '100', '50ktne': '75', '50ktse': '30', '50ktsw': '30', '50ktnw': '50', '64ktne': '30', '64ktse': '0', '64ktsw': '20', '64ktnw': '20', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20051205, 'time': '1800', 'category': 'HU', 'lt': '33.7N', 'lg': '34.6W', 'maxwind': '70', 'maxpres': '985', '34ktne': '75', '34ktse': '75', '34ktsw': '75', '34ktnw': '100', '50ktne': '75', '50ktse': '30', '50ktsw': '30

IndexError: index exceeds dimension bounds